<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Machine_Perception_with_Flowers_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Hello guys
In this Notebook, A Classifier is implemented in TensorFlow to classify images of [Flowers dataset](https://www.tensorflow.org/datasets/catalog/tf_flowers) from tensorflow, It contains 3670 images falling into 5 categories:


*   Daisy
*   Roses
*   Dandelion
*   SunFlowers
*   Tulips

you can access it from here:

```gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/```



In [1]:
import tensorflow as tf

In [2]:
print(tf.version.VERSION)

2.9.2


In [5]:
gpu_name = tf.test.gpu_device_name()
print(f'The GPU found at: {gpu_name}')

The GPU found at: 
